In [1]:
import pandas as pd
import numpy as np
import math
from math import sin, cos, sqrt, atan2
from matplotlib import pyplot as plt
import numpy as np
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.feature_selection import RFECV
import warnings
import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler
plt.style.use('seaborn')

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 100)

In [2]:
df=pd.read_csv('./data/survey.csv')

In [3]:
df.columns=[x.lower() for x in df.columns]

In [4]:
df.drop(list(df[df.self_employed=='Yes'].index),inplace=True)
df.drop('comments',axis=1,inplace=True)

In [5]:
df['timestamp']=[" ".join(x[:-9].split('-')).replace(" ","-") for x in df.timestamp]

In [6]:
# could maybe do it like this to capture all gender possibilities
# make all answers lowercase
df.gender = df.gender.str.lower()
# create three lists male, female and other of all the entries
male = ["male", "m", "male-ish", "maile", "mal", "male (cis)", "make", "male ", "man","msle", "mail", "malr","cis man", "cis male"]
female = ["cis female", "f", "female", "woman", "femake", "female ","cis-female/femme", "female (cis)", "femail"]
gender_variant = ["trans-female", "something kinda male?", "queer/she/they", "non-binary","nah", "all", "enby", "fluid", 
     "genderqueer", "androgyne", "agender", "male leaning androgynous", "guy (-ish) ^_^", "trans woman", "neuter", 
     "female (trans)", "queer", "ostensibly male, unsure what that really means", "p", "a little about you"]
# use .loc and .isin to map male, female and gender variant to
df.gender.loc[df.gender.isin(male)] = 'male'
df.gender.loc[df.gender.isin(female)] = 'female'
df.gender.loc[df.gender.isin(gender_variant)] = 'gender_variant'

/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
conditions=[
    df.no_employees=="1-5",            
    df.no_employees=="6-25",         
    df.no_employees=="26-100", 
    df.no_employees=="100-500",       
    df.no_employees=="500-1000" ,           
    df.no_employees=="More than 1000",
]
choices=[
    "extra_small",
    "small",
    "medium",
    "medium_large",
    "large",
    "extra_large",
    
]
df['no_employees']=np.select(conditions,choices,default=0)

In [8]:
conditions=[
    (df["mental_health_consequence"].str.lower()=='yes'),            
    (df["mental_health_consequence"].str.lower()=='no'),
    (df["mental_health_consequence"].str.lower()=='maybe')|(df["mental_health_consequence"].str.lower()=='don\'t know')|(df["mental_health_consequence"].str.lower()=='not sure'),
]
choises=[
    1,
    0,
    1,

]
df["mental_health_consequence"]=np.select(conditions,choises,default=0)

In [9]:
df.work_interfere=df.work_interfere.fillna("Missing")
df.self_employed=df.self_employed.fillna("Missing")

In [10]:
north=['Canada','United States']
uk=['United Kingdom']
eu=['Germany','Ireland','Netherlands','France','Italy','Belgium','Spain','Norway']
for x in df.country.index:
    if df.country[x]in north:
        df.country[x]= "North America"
    elif df.country[x]in uk:
        df.country[x]= "United Kingdom"
    elif df.country[x]in eu:
        df.country[x]= "Europe"
    else:
        df.country[x]= 'Other'

/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/carlosruiz/opt

In [11]:
north= ['CT', 'DE', 'IL', 'IN', 'IA', 'KS', 'ME', 'MA', 'MD', 'MI', 'MN', 'MO', 'NE','NH', 'NJ', 'NY', 'ND', 'OH', 'PA', 'RI', 'SD', 'VT', 'WI','DC']
south= ['AL', 'AR', 'FL', 'GA', 'KT', 'LA', 'MS', 'NC', 'OK', 'SC', 'TN', 'TX', 'VA', 'WV', 'PR']
west= ['AK', 'AZ', 'CA', 'CO', 'HI', 'ID', 'MT', 'NV', 'NM', 'OR', 'UT', 'WA', 'WY']


for x in df.state.index:
    if df.state[x]in north:
        df.state[x]= "Northern State"
    elif df.state[x]in south:
        df.state[x]= "Southern State"
    elif df.state[x]in west:
        df.state[x]= "Western State"
    else:
        df.state[x]= 'Non US'

/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we lo

In [12]:
df=pd.get_dummies(df,columns=['no_employees','gender','country','state','self_employed','family_history','treatment','work_interfere','remote_work','tech_company','benefits','care_options','wellness_program','seek_help','anonymity','leave','phys_health_consequence','coworkers','supervisor','mental_health_interview','phys_health_interview','mental_vs_physical','obs_consequence'],)


In [13]:
df.drop(["state_Non US",],axis=1,inplace=True)

In [14]:
# Function to find outliers in Age column
def outliers(df):
    '''Takes in a dataframe and returns a list of outliers > 99th and < 1 percentile'''
    outliers = []
    for i in df:
        q1 = np.percentile(df, .5)
        q3 = np.percentile(df, 99.5)
        if i > q3 or i < q1:
            outliers.append(i)
    print("Outliers:", outliers)
    return outliers

In [15]:
age_outliers = outliers(df['age'])

Outliers: [-29, 60, 329, 58, 62, -1726, 5, 72, 60]


In [16]:
df = df.loc[~df['age'].isin(age_outliers)]

In [27]:
df

,timestamp,age,mental_health_consequence,no_employees_extra_large,no_employees_extra_small,no_employees_large,no_employees_medium,no_employees_medium_large,no_employees_small,gender_female,gender_gender_variant,gender_male,country_Europe,country_North America,country_Other,country_United Kingdom,state_Northern State,state_Southern State,state_Western State,self_employed_Missing,self_employed_No,family_history_No,family_history_Yes,treatment_No,treatment_Yes,work_interfere_Missing,work_interfere_Never,work_interfere_Often,work_interfere_Rarely,work_interfere_Sometimes,remote_work_No,remote_work_Yes,tech_company_No,tech_company_Yes,benefits_Don't know,benefits_No,benefits_Yes,care_options_No,care_options_Not sure,care_options_Yes,wellness_program_Don't know,wellness_program_No,wellness_program_Yes,seek_help_Don't know,seek_help_No,seek_help_Yes,anonymity_Don't know,anonymity_No,anonymity_Yes,leave_Don't know,leave_Somewhat difficult,leave_Somewhat easy,leave_Very difficult,leave_Very easy,phys_health_consequence_Maybe,phys_health_consequence_No,phys_health_consequence_Yes,coworkers_No,coworkers_Some of them,coworkers_Yes,supervisor_No,supervisor_Some of them,supervisor_Yes,mental_health_interview_Maybe,mental_health_interview_No,mental_health_interview_Yes,phys_health_interview_Maybe,phys_health_interview_No,phys_health_interview_Yes,mental_vs_physical_Don't know,mental_vs_physical_No,mental_vs_physical_Yes,obs_consequence_No,obs_consequence_Yes
0,2014-08-27,37,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,1,0
1,2014-08-27,44,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0
2,2014-08-27,32,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0
3,2014-08-27,31,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1
4,2014-08-27,31,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2015-09-12,26,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
1255,2015-09-26,32,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0
1256,2015-11-07,34,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0
1257,2015-11-30,46,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0


In [31]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [32]:
# create columns for years, months, days, dow, hour to see when majority of surveys were administered
df['Year'] = df['timestamp'].dt.year
df['Month'] = df['timestamp'].dt.month
df['Day'] = df['timestamp'].dt.day
df['Dow'] = df['timestamp'].dt.dayofweek


In [34]:
x=df.drop(['timestamp','mental_health_consequence'],axis=1)
y=df.mental_health_consequence

In [35]:
X_train,y_trian,X_test,y_test

,age,no_employees_extra_large,no_employees_extra_small,no_employees_large,no_employees_medium,no_employees_medium_large,no_employees_small,gender_female,gender_gender_variant,gender_male,country_Europe,country_North America,country_Other,country_United Kingdom,state_Northern State,state_Southern State,state_Western State,self_employed_Missing,self_employed_No,family_history_No,family_history_Yes,treatment_No,treatment_Yes,work_interfere_Missing,work_interfere_Never,work_interfere_Often,work_interfere_Rarely,work_interfere_Sometimes,remote_work_No,remote_work_Yes,tech_company_No,tech_company_Yes,benefits_Don't know,benefits_No,benefits_Yes,care_options_No,care_options_Not sure,care_options_Yes,wellness_program_Don't know,wellness_program_No,wellness_program_Yes,seek_help_Don't know,seek_help_No,seek_help_Yes,anonymity_Don't know,anonymity_No,anonymity_Yes,leave_Don't know,leave_Somewhat difficult,leave_Somewhat easy,leave_Very difficult,leave_Very easy,phys_health_consequence_Maybe,phys_health_consequence_No,phys_health_consequence_Yes,coworkers_No,coworkers_Some of them,coworkers_Yes,supervisor_No,supervisor_Some of them,supervisor_Yes,mental_health_interview_Maybe,mental_health_interview_No,mental_health_interview_Yes,phys_health_interview_Maybe,phys_health_interview_No,phys_health_interview_Yes,mental_vs_physical_Don't know,mental_vs_physical_No,mental_vs_physical_Yes,obs_consequence_No,obs_consequence_Yes,Year,Month,Day,Dow,Hour
0,37,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,1,0,2014,8,27,2,0
1,44,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,2014,8,27,2,0
2,32,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,2014,8,27,2,0
3,31,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,2014,8,27,2,0
4,31,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,2014,8,27,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,26,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,2015,9,12,5,0
1255,32,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,2015,9,26,5,0
1256,34,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,2015,11,7,5,0
1257,46,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,2015,11,30,0,0


## Logistic Regression

In [40]:
x=df.drop(['timestamp','mental_health_consequence'],axis=1)
y=df.mental_health_consequence

In [41]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=2020,test_size=0.2)

In [42]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(class_weight='balanced')
lr.fit(X_train,y_train)

/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(class_weight='balanced')

In [43]:
y_pred_lr = lr.predict(X_test)

In [44]:
from sklearn import metrics
print('Logistic Regression Accuracy: {}'.format(metrics.accuracy_score(y_test, y_pred_lr)))
print('Logistic Regression Accuracy: {}'.format(metrics.f1_score(y_test,y_pred_lr)))

Logistic Regression Accuracy: 0.8461538461538461
Logistic Regression Accuracy: 0.8768115942028984


In [49]:
coef=list(lr.coef_[0])
col=x.columns
comb={}
for i in range(len(coef)):
    comb[col[i]]=coef[i]
comb=pd.DataFrame(comb,index=[0]).T
comb.columns=['Coeficients']
comb=comb.sort_values(ascending=False,by='Coeficients').head(20)
comb.head(5)

,Coeficients
phys_health_consequence_Maybe,1.427658
supervisor_No,1.093694
mental_vs_physical_No,0.667647
leave_Very difficult,0.580770
work_interfere_Often,0.504404


### Logistic Regression with Recursive Feature Elimination

In [56]:
from sklearn.feature_selection import RFECV

lr = LogisticRegression()
selector = RFECV(estimator=lr, step=1, cv=5, scoring='f1', n_jobs=-1)
selector.fit(X_train, y_train)
rfe_features = x.columns[(selector.get_support())]


/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modu

In [57]:
X = df[rfe_features]
Y= y.copy()

In [58]:
lr2=LogisticRegression(class_weight="balanced")

In [59]:
X_train2,X_test2,y_train2,y_test2=train_test_split(X,Y,random_state=2020, test_size=.2)

In [60]:
lr2.fit(X_train2,y_train2)

y_predict_lr2=lr2.predict(X_test2)

In [63]:
print('Logistic Regression Accuracy: {}'.format(metrics.accuracy_score(y_test2, y_predict_lr2)))
print('Logistic Regression Accuracy: {}'.format(metrics.f1_score(y_test2,y_predict_lr2)))

Logistic Regression Accuracy: 0.8687782805429864
Logistic Regression Accuracy: 0.8953068592057761


In [62]:
coef2=list(lr2.coef_[0])
col2=X.columns
comb2={}
for i in range(len(coef2)):
    comb2[col2[i]]=coef2[i]
comb2=pd.DataFrame(comb2,index=[0]).T
comb2.columns=['Coeficients']
comb2=comb2.sort_values(ascending=False,by='Coeficients')
comb2.head(5)

,Coeficients
leave_Very difficult,1.316006
phys_health_consequence_Maybe,1.079535
supervisor_No,1.030822
work_interfere_Often,1.019595
mental_health_interview_No,0.822508


## Final Results

In [64]:
print('Without RFE')
print('Logistic Regression Accuracy: {}'.format(metrics.accuracy_score(y_test, y_pred_lr)))
print('Logistic Regression Accuracy: {}'.format(metrics.f1_score(y_test,y_pred_lr)))
print('With RFE')
print('Logistic Regression Accuracy: {}'.format(metrics.accuracy_score(y_test2, y_predict_lr2)))
print('Logistic Regression Accuracy: {}'.format(metrics.f1_score(y_test2,y_predict_lr2)))

Without RFE
Logistic Regression Accuracy: 0.8461538461538461
Logistic Regression Accuracy: 0.8768115942028984
With RFE
Logistic Regression Accuracy: 0.8687782805429864
Logistic Regression Accuracy: 0.8953068592057761


In [72]:
print(comb.head(10))
print('\n')
print(comb2.head(10))

                               Coeficients
phys_health_consequence_Maybe     1.427658
supervisor_No                     1.093694
mental_vs_physical_No             0.667647
leave_Very difficult              0.580770
work_interfere_Often              0.504404
coworkers_No                      0.450971
mental_health_interview_No        0.443743
benefits_Yes                      0.433810
state_Southern State              0.414000
phys_health_interview_Yes         0.333065


                               Coeficients
leave_Very difficult              1.316006
phys_health_consequence_Maybe     1.079535
supervisor_No                     1.030822
work_interfere_Often              1.019595
mental_health_interview_No        0.822508
anonymity_No                      0.814032
phys_health_consequence_Yes       0.805782
benefits_Yes                      0.761727
country_United Kingdom            0.711202
state_Southern State              0.675427
